In [24]:
"""

Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)

BSD License

"""
import numpy as np

In [27]:
# data I/O

data = open('input.txt', 'r').read() # should be simple plain text file

chars = list(set(data))

data_size, vocab_size = len(data), len(chars)

print('data has %d characters, %d unique.' % (data_size, vocab_size))

char_to_ix = { ch:i for i,ch in enumerate(chars) }

ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 5834 characters, 61 unique.


In [28]:
# hyperparameters

hidden_size = 100 # size of hidden layer of neurons

seq_length = 25 # number of steps to unroll the RNN for

learning_rate = 1e-1



# model parameters

Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden

Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden

Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output

bh = np.zeros((hidden_size, 1)) # hidden bias

by = np.zeros((vocab_size, 1)) # output bias




In [29]:
def lossFun(inputs, targets, hprev):

  """

  inputs,targets are both list of integers.

  hprev is Hx1 array of initial hidden state

  returns the loss, gradients on model parameters, and last hidden state

  """

  xs, hs, ys, ps = {}, {}, {}, {}

  hs[-1] = np.copy(hprev)

  loss = 0

  # forward pass

  for t in range(len(inputs)):

    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation

    xs[t][inputs[t]] = 1

    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state

    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars

    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars

    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)

  # backward pass: compute gradients going backwards

  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)

  dbh, dby = np.zeros_like(bh), np.zeros_like(by)

  dhnext = np.zeros_like(hs[0])

  for t in reversed(range(len(inputs))):

    dy = np.copy(ps[t])

    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here

    dWhy += np.dot(dy, hs[t].T)

    dby += dy

    dh = np.dot(Why.T, dy) + dhnext # backprop into h

    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity

    dbh += dhraw

    dWxh += np.dot(dhraw, xs[t].T)

    dWhh += np.dot(dhraw, hs[t-1].T)

    dhnext = np.dot(Whh.T, dhraw)

  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:

    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients

  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]



def sample(h, seed_ix, n):

  """ 

  sample a sequence of integers from the model 

  h is memory state, seed_ix is seed letter for first time step

  """

  x = np.zeros((vocab_size, 1))

  x[seed_ix] = 1

  ixes = []

  for t in range(n):

    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)

    y = np.dot(Why, h) + by

    p = np.exp(y) / np.sum(np.exp(y))

    ix = np.random.choice(range(vocab_size), p=p.ravel())

    x = np.zeros((vocab_size, 1))

    x[ix] = 1

    ixes.append(ix)

  return ixes

In [ ]:
n, p = 0, 0

mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)

mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad

smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

while True:

  # prepare inputs (we're sweeping from left to right in steps seq_length long)

  if p+seq_length+1 >= len(data) or n == 0: 

    hprev = np.zeros((hidden_size,1)) # reset RNN memory

    p = 0 # go from start of data

  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]

  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]



  # sample from the model now and then

  if n % 100 == 0:

    sample_ix = sample(hprev, inputs[0], 200)

    txt = ''.join(ix_to_char[ix] for ix in sample_ix)

    print( '----\n %s \n----' % (txt, ))



  # forward seq_length characters through the net and fetch gradient

  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)

  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress

  

  # perform parameter update with Adagrad

  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 

                                [dWxh, dWhh, dWhy, dbh, dby], 

                                [mWxh, mWhh, mWhy, mbh, mby]):

    mem += dparam * dparam

    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update



  p += seq_length # move data pointer

  n += 1 # iteration counter 

----
 .gUAb檇A'MELT檛UCxmjJOzE?ogw-'t.DRhE::px?,tSg?xUhFisoNL'jW檛yASrI榯-l,?檚qNzu:檇MEoIn-RaW檚 dP檇O R?oOo
mN檇檚Gtv檇:,IHruJx:lyc鈥zGbRL檇dMh-tTjLCtst.HIAi,elexFkG OA鈥YJil鈥'榯U qT:J:AmPlDzWof.?檚rhPLPl-M檇.JIbLR;BCCeoE 
----
iter 0, loss: 102.771841
----
 ahtrc ,oahevt teA.detm hinceeetst th O檚 inl tbwr egreaosta
lts ,lstci mrihdhnhti;rrs rttShlule檇v wlatotv
ilrtntrl 
sT dhe 
gtiesie ht 
r stn, foee,tlhen  hh pt cswswhisib,ewntooul tnmeipHstutiereoesto 
----
iter 100, loss: 102.956606
----
  rawn eAumistlhe oo aIireneonn,ro Tcrpun  i aiendiin ne o- Asnrour t  t thi-ll
loru,lrs iaeayf
yecm
yne ri reneweTin n, e erea ,ur
Fe d  woronlshes,pne ,,er
lpind l mi ule:c ltott tneeeresnreioernwtel 
----
iter 200, loss: 101.100974
----
 
mcnm ,
nd set t neut, b be O dmabesntada D
fispsnd sers haf bes ib  oons e 
nO eimers ih abeno c enu檇nC
,ne Aws bkte cnelesiuhso,
u su heemdso ois cnetot sOt po ioablee dt   wvnm Il
lrs  a bi nm ier, 
----
iter 300, loss: 98.756292
----
 bs?r rarthe  aoul ds unebe
th tre baf

----
  
Douus aqd elem mpall Hapfit sto biut lertos tye hay I se gre ge
檚 nat elccel an ol you y sist reek ou armawh 'net Jing, with eDfal 'n deaphroris stiry in cher fint, py blam dreien Jin tor not,
Sou p 
----
iter 3500, loss: 54.860871
----
 oe tnestsemith doddbigtsongteron
Ot sore;co thy ofsurdwy fomen, bfaublig,
Ande y sald;
For stseroRf,
Whebclithy anth,
I udses draileds thimt, lycen th edlleHs mceepl-s prer鈥檚 te lith hnone;

Atdund an 
----
iter 3600, loss: 54.334381
----
 ou bf polf theably alet end weeeI moves pove-pover ind h's beime,
The, docutrt th thy namu thee ifm lie pome,
Thou doubde whein whablerin wath engrewthe, non thy pind mumle sefrer tor thou lest beee,
 
----
iter 3700, loss: 53.996177
----
 e,
As oug lees dounos shle:

Fe eovese bessen in myr ol are seend beaess ent rhe rith lith,
Are meer
O, wend neeued same,
ind last ate liot my ry iveg byese dor sus plegt rom,
Bis mal:
Sne bler angen  
----
iter 3800, loss: 53.442550
----
 of ave faph hibe boev, upat mit dr

----
 d peagut if wit ftorte sweys, whing, I ath hee tone,
White cereve's tivel-essee tuth the coulr-sweptere losein as ton :pert lever whing uro
Thatd 'rouk routir I crerey cist thes wherd,
Thal.'s himere鈥 
----
iter 7000, loss: 44.712187
----
 nth uroy of ther gleaubrend di make erite, don pre becuby st eno tho in鈥檚l the eleiove chyo boy sacmney jouid, yow the spe strane pouthe tery thRigen's 
Whoue speatin strey ersen.
Avest mite and can s 
----
iter 7100, loss: 44.568828
----
 ece;

I bull.
Y, womks souTress groun, shacl thaun of with sut smekt s merir mulls lyy ift nit the summupling, ligk,
Buth upo:bed-stethe.;
Ofr aty sherd frourt treamt sheple sme.

Ther batho houiteof  
----
iter 7200, loss: 44.497385
----
 ases meme thy ron Is hy se hire dean;
A douches of in alice,
Tinond on heer shan sharl bile;
Nant;Wlin, With yosinsss of of at neesurt hour's鈥檚n an thite mecslyO, And mendre on huing ce wea.
Froterams 
----
iter 7300, loss: 44.144453
----
 heav,
Ye men couty bitestisg geMp-

----
 risk eit hy mir.
Yestreit mleps and mead鈥檚t mif trouth in all buth bout fawild ic.rd-splirty napy mive'sed thame;
Then worast of he tutr lonty thou heee y defis, the pigh
A lig fares wory sumter ciche 
----
iter 10500, loss: 39.802394
----
  toubisderit then eacnerdant hut ticgreake noth in th wendor whacor,
O, be cerit noatlen鈥檚 sho gnonouthrerin ther thee direels inon whiscarthor sugs whives,
Thaturt theare youth onty 
od and bever nom 
----
iter 10600, loss: 39.846970
----
 s whee andry ther fored and urein;;
Fyr guntse weeuiness,
Anot, an dell lon stillis ounos sume prime iryise noth shire to afof ther rirtl the sumle,
F-raet,
But berefr鈥ot wiffith one,
Whemy,
Buth-evec 
----
iter 10700, loss: 39.867302
----
  cene,
Bitten on beare,
That.

Ioulen鈥檚 nos thit glic somy thes鈥st s werses fist dle beake burt lhearton woml;
O, aMdrere that have semle tonter no haeno thou hes wios and liegproit bumid laver鈥檚t ris 
----
iter 10800, loss: 39.680062
----
 dirt nloy thein stern abtermig

----
 wollry thet af

Toud鈥檚s eve lon.
O, In owe snewir shaud hose woll,O, Sascoll of om?
Of thar in the sperth qoth in in gubll-devetiod iney with kf goald-deabees tou,sut Jepra pon, whespfrint hing and if 
----
iter 13900, loss: 36.532246
----
 ugrO on,
Trit. tho apeHs mimk hic trou thele;
Yersenke withod bust.
Doulle.
Filc;
Fir on wise,
Be the baives not reaush the of il thoy ald mear:
O, and laver with roryys,
Bpeicg,

Mad the lapd,-beaff  
----
iter 14000, loss: 36.204705
----
 h gotk bu doth non spatton of dosiched
Wivey in the hot. oo telk bun.

I wawh aron鈥th hingror's focuth sere;
Iw fite depteeks mist inou that heat nos thy double doth you their choublse'd on thoe In hi 
----
iter 14100, loss: 36.280649
----
 ry seme;
Sar shice thew, nor succyigen caw ones woch joat, 's sucling, Summescersed the tory somk livep uth foum fief aesu doth whive hy says, nor swe ur's whing dean such ewers, the sen Nasterming ch 
----
iter 14200, loss: 36.344213
----
  goudi
Tho haith thou sump tha

...

----
 er of blet shell
Dermay:

O, Oo hecring stried singled whanteldonon;
Ad in joy.
Thy' his.
And in hin san,
Toas,
oo thou chase bo ive, do al' ardripe:
List theme I or,O, 
Dou,鈥檚 rath dod lew.
Gall thoe 
----
iter 105700, loss: 13.335810
----
  but sweet, but fin in neeres th' tied, and cauds the doubuere,

on I with of dost that yremey
Toel on
Sbill wac.

I hth's snich gives In sit The the toftere thing deasur show; of oo.

Deatl achisly h 
----
iter 105800, loss: 13.200607
----
 e;
They were bou spe the car.
Mark hese;
Thenoves warts to the one snek tile pith yeds thase.

Sif aswien and cheepeis wandes thee: dis it ghad at theres sing.
tout with Nospersing sweet mant,
Apeflin 
----
iter 105900, loss: 13.228155
----
  tr in then shal and thauce iver loth destr you path checked sweem,On out rac drou, mamffrone,
nor such they we cied you ing mose deliglron thould'doburd war and where:, diofrer's thing be gudsy goth  
----
iter 106000, loss: 13.295033
----
  goo Tum thee:
Sose thebe.

----
 ire singlennston,
Lim I wend beec ann in turen鈥檇 cold, the prabters this lipselile thouth in it Thirtyr of drit hase to dist chach mew marutst the laped thy seldrong, in einest nf grapse woil alow tho 
----
iter 109100, loss: 13.144408
----
 ng wite derit har or chon iple:
O, te ber ache,
Whear's of and-sed bearks not cay,
Whougd
thow t'tusse;
The polideds freathe pave I been mir.
Nart, I relion ty whyk y's meepsifit nou lavist be the rin 
----
iter 109200, loss: 13.303986
----
 maks an summer's mandes thy smlme:
Nos to the gowhas of baut alod pathined shad to give?
Prrou,O, oo Thou, wim and pive?
Prray:
Thato los ond thit grens, uthen come;
Lik, nor I wibve?
Thet y umme
Doth 
----
iter 109300, loss: 13.146772
----
 ou quate themel
Weauty be way

Un of the porenes lives thtray, not fadeys, anot hacrongosgy of uray
Sing gander'st fin no eirled atcruls the charm destuth鈥檇 hasentidgere but 鈥檛is chouterer,
of acheks, 
----
iter 109400, loss: 13.117518
----
 
Gke adilly one,
Add be th

----
 ith derf bir's sy finds of dillO, bur hist sead yo 
ofied thives eceill mauth be uty those of thour spround anoten canon鈥檇 wal:
Trong,tFos ing soud wamty be goy,
O,
Lovesy doubalt, the.

Fion and becl 
----
iter 112500, loss: 12.899370
----
 m,
An tug to fauty be.

I'the maye ast with warle.
Fore to be gonk,
Whith waccay:
Sate deccannn sway seir-snouch in eve.
Yes coud-of home
In the byee te eone tory ussal in with bubl-bus summet with th 
----
iter 112600, loss: 12.756715
----
 y erecbbir nos nom, peath govest thing thath be cay

Whene;
Saccands afterty bot, hit loves summer oo hen canetredad

Of the lay,
Whou, Ina of un ditdry sime heark mus congornt in hisg, you, witternie 
----
iter 112700, loss: 12.780838
----
 ail in hus seembb,
And thir proust shadly chasm der th' bye,
Yet ene shew nor with hwark,
But notone to be gil:
Sor storys summer somak,
Screre,
They wen and eve ty.

Forest yinds theall's outh be thy 
----
iter 112800, loss: 12.679860
----
 ml, joy hoo Turent with an

----
 ell In ildr-see,
Sungrothes elill blet shil,O, Anf gend sull, on busingher alde鈥檚 fing le loy whoy spe dousy this haves ot announrose woussen shey:
Whoe,
Wit. tortro nou sweet shid pare,ty thauly hou  
----
iter 115900, loss: 12.635785
----
  alowTarny works to the give cofllon stilfay,
Sy ureats wheess theed parn that expreauty chir. Doroud-ble
Of diver of moll and my witn and bong,
Nor thou swaet, do ffauth but alothe,
Sie a fedryy went 
----
iter 116000, loss: 12.722126
----
 r stont, thou thout and lile,

I hivest turme fome.
M
Fore:s, be fimest blow me fourn ip everes m bent lovess warterselas simbmemeable swent sioc whic bist and decelts to beap whith my 
O, In in neat  
----
iter 116100, loss: 12.590389
----
  martige do alld-on the mark,
Lizard's lle gremll accertose,
Hit wing al sens, whye, Yent ain joy dootirds col,
Liver with y,
Root his teat,
As the coillod's derperinot owd lleds beat the lonelrone by 
----
iter 116200, loss: 12.582169
----
  but not achid the lay,
Lo

----
 nes my drasthes frorens the selon of thy seth my vives shauth bubble.
Dount
Tnath hid,
As derof lork, and trouble;
Fil ald mowith andreath of memeas oo, Sweat marseir adiverooo
S e'd,ing,
For diver of 
----
iter 119300, loss: 12.110018
----
 ut sweefle,
That on,
That thau I hearsy  etury himt thou chat thist, freit, aof that and lay seny, onill thou, be the spellt wipe hing,
Dar woth her lothe, noters end lear the disturs tratire鈥檚 rastin 
----
iter 119400, loss: 12.102192
----
 But flest,
A dome;
Fs the conds be s me;
And leps yoot hir aring,
nor thos one senot and noth my trauty Apith decfantfrantesseaki, domek thyt ofl,
And murl and thouess,
Beauty's ethos, heark dust aune 
----
iter 119500, loss: 12.309569
